In [1]:
import math
import random

def fabs(x):
    return abs(x)

def sqrt(x):
    return x**0.5

def exp(x):
    return math.e**x

def pow(x, y):
    return x**y

def log(x, base):
    return math.log(x, base)

def fmod(x, y):
    return x % y

def ceil(x):
    return math.ceil(x)

def floor(x):
    return math.floor(x)

def trunc(x):
    return math.trunc(x)

# define constants
pi = math.pi
e = math.e

def radians(x):
    return x * pi / 180

def degrees(x):
    return x * 180 / pi

def sin(x):
    return math.sin(x)

def cos(x):
    return math.cos(x)

def tan(x):
    return math.tan(x)

def asin(x):
    return math.asin(x)

def acos(x):
    return math.acos(x)

def atan(x):
    return math.atan(x)

def atan2(y, x):
    return math.atan2(y, x)

def random():
    return random.random()

def uniform(a, b):
    return random.uniform(a, b)

def randint(a, b):
    return random.randint(a, b)

def choice(seq):
    return random.choice(seq)

def randrange(start, stop=None, step=1):
    return random.randrange(start, stop, step)

def seed(x=None):
    random.seed(x)



In [2]:
from inbuilt import *
from dsp import *
import numpy as np # To verify some things work

In [3]:
# Simple convolution
def convolve(x, y):
    z = []
    for i in range(len(x) + len(y) - 1):
        z.append(0)
    for i in range(len(x)):
        for j in range(len(y)):
            z[i + j] += x[i] * y[j]
    return z

In [4]:
# Lets convolve two signals: [0, 1, 2, 3, 4] and [0, 1, 2, 3, 4] first using the dsp.py and then using numpy.convolve
x_signal = np.array([0, 1, 2, 3, 4])
y_signal = np.array([0, 1, 2, 3, 4])
print("Numpy Convolution: ", np.convolve(x_signal, y_signal))
print("DFT Convolution: ", convolve(x_signal, y_signal))

Numpy Convolution:  [ 0  0  1  4 10 20 25 24 16]
DFT Convolution:  [0, 0, 1, 4, 10, 20, 25, 24, 16]


In [33]:
# round does not work for complex numbers so we will have to define our own function
def rounds(x, n):
    if isinstance(x, complex):
        return complex(round(x.real, n), round(x.imag, n))
    else:
        return round(x)
    
# We are going to make a special cexp function that will return the complex exponential of a complex number
def cexp(x):
    return complex(exp(x.real) * cos(x.imag), exp(x.real) * sin(x.imag))  

# Simple FFT
def FFT(f, N=None):
    if N is None:
        N = len(f)
    # We will use a recursive implementation of the FFT and return the coefficients and the number of operations
    operations = 0
    if N == 1:
        return f, operations
    else:
        # We do not know if the length of f is a power of 2, so we will pad it with zeros
        # We will pad it with zeros until it is a power of 2
        while N & (N - 1) != 0:
            f.append(0)
            N += 1
        # We will split the coefficients into even and odd
        Y_even, operations_even = FFT(f[::2], N//2)
        Y_odd, operations_odd = FFT(f[1::2], N//2)
        operations = operations_even + operations_odd
        # factor = exp(-2j * pi * arange(N) / N)
        # this gives us an error since we cannot multiply a complex number by an array since we are not using numpy
        # so instead we will have to loop through the array and multiply each element by the complex number
        factor = []
        for i in range(N):
            factor.append(cexp(-2j * pi * i / N))
        # Y = concatenate([Y_even + factor[:N//2] * Y_odd, Y_even + factor[N//2:] * Y_odd])
        # this gives us an error since we cannot multiply a complex number by an array since we are not using numpy
        # so instead we will have to loop through the array and multiply each element by the complex number
        Y = []
        # Lets also make sure that we are rounding to 3 decimal places
        for i in range(N//2):
            Y.append(rounds(Y_even[i] + factor[i] * Y_odd[i], 5))
        for i in range(N//2):
            Y.append(rounds(Y_even[i] + factor[i + N//2] * Y_odd[i], 5))
        operations += N
        return Y, operations
    
# Inverse FFT
def IFFT(f, N=None):
    if N is None:
        N = len(f)
    # We will use a recursive implementation of the FFT and return the coefficients and the number of operations
    operations = 0
    if N == 1:
        return f, operations
    else:
        # We do not know if the length of f is a power of 2, so we will pad it with zeros
        # We will pad it with zeros until it is a power of 2
        while N & (N - 1) != 0:
            f.append(0)
            N += 1
        # We will split the coefficients into even and odd
        Y_even, operations_even = IFFT(f[::2], N//2)
        Y_odd, operations_odd = IFFT(f[1::2], N//2)
        operations = operations_even + operations_odd
        # factor = exp(-2j * pi * arange(N) / N)
        # this gives us an error since we cannot multiply a complex number by an array since we are not using numpy
        # so instead we will have to loop through the array and multiply each element by the complex number
        factor = []
        for i in range(N):
            factor.append(cexp(2j * pi * i / N))
        # Y = concatenate([Y_even + factor[:N//2] * Y_odd, Y_even + factor[N//2:] * Y_odd])
        # this gives us an error since we cannot multiply a complex number by an array since we are not using numpy
        # so instead we will have to loop through the array and multiply each element by the complex number
        Y = []
        # Lets also make sure that we are rounding to 3 decimal places
        for i in range(N//2):
            Y.append(rounds(Y_even[i] + factor[i] * Y_odd[i], 5))
        for i in range(N//2):
            Y.append(rounds(Y_even[i] + factor[i + N//2] * Y_odd[i], 5))
        operations += N
        return Y, operations
    
# DFT
def DFT(f, N=None):
    if N is None:
        N = len(f)
    result = []
    for k in range(N):
        sum = 0
        for n in range(len(f)):
            sum += complex(f[n]) * cexp(-1j * 2 * pi * k * n / N)
        result.append(rounds(sum, 5))
    return result

# IDFT
def IDFT(f, N=None):
    if N is None:
        N = len(f)
    result = []
    for k in range(N):
        sum = 0
        for n in range(len(f)):
            sum += complex(f[n]) * cexp(1j * 2 * pi * k * n / N)
        result.append(rounds(sum / N, 5))
    return result

# Simple convolution
def convolve(x, y):
    z = []
    for i in range(len(x) + len(y) - 1):
        z.append(0)
    for i in range(len(x)):
        for j in range(len(y)):
            z[i + j] += x[i] * y[j]
    return z


In [40]:
# np convolve [1,2] with itself
x_signal = np.array([1, 2])
y_signal = np.array([1, 2])
print("Numpy Convolution: ", np.convolve(x_signal, y_signal))

Numpy Convolution:  [1 4 4]


In [34]:
# Test the FFT
x = [1.55, 2.5, 3.2, 4.69]
print("FFT: ", FFT(x, len(x)))
# Numpy FFT
print("Numpy FFT: ", np.fft.fft(x))

FFT:  ([(11.94+0j), (-1.65+2.19j), (-2.44-0j), (-1.65-2.19j)], 8)
Numpy FFT:  [11.94+0.j   -1.65+2.19j -2.44+0.j   -1.65-2.19j]


In [39]:
DFT(x, len(x))
# take the inverse FFT using Numpy
print("Numpy IFFT: ", np.fft.ifft(np.fft.fft(x)))
print("IFFT: ", (IFFT(DFT(x, len(x)), len(x))[0]))
print("IDFT: ", IDFT(DFT(x, len(x)), len(x)))

Numpy IFFT:  [1.55+0.j 2.5 +0.j 3.2 +0.j 4.69+0.j]
IFFT:  [(6.2+0j), (10+0j), (12.8-0j), (18.76-0j)]
IDFT:  [(1.55+0j), (2.5+0j), (3.2+0j), (4.69-0j)]
